In [1]:
import pytesseract
from PIL import ImageGrab
import time
from tkinter import *
import tkinter.font as tkFont
from openai import OpenAI
import base64
import io

class Api:
    def __init__(self,api,role):
        self.model = api["model"]
        self.api_key = api["api_key"]
        self.base_url = api["base_url"]
        self.is_steam = api["is_steam"]
        self.role = role

        self.client = OpenAI(api_key=self.api_key, base_url=self.base_url)

    def talk(self,question):

        response = self.client.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": self.role},
                {"role": "user", "content": question}
            ],
            stream=self.is_steam
        )
        answer = response.choices[0].message.content
        return answer
    

    def talk_img(self,question,img=None):

        file = f"data:image;base64,{img}"

        response = self.client.chat.completions.create(
            model=self.model,
            messages=[

                {"role": "system", 
                 "content": [
                     {"type":"text","text": self.role}
                ]},

                {"role": "user",
                 "content": [
                    {"type": "text","text": question},
                    {"type": "image_url","image_url": {"url":file},}
                ]}
            ]
        )
        
        answer = response.choices[0].message.content
        return answer

qwen_plus_api = {

    "model":"qwen-plus",
    "api_key":"sk-e30aaee8bcce40348cf790585352bf36",
    "base_url":"https://dashscope.aliyuncs.com/compatible-mode/v1",
    "is_steam":False
    
}

# 传图片ocr
qwen_vl_ocr_plus_api = {

    "model":"qwen-vl-ocr-latest",
    "api_key":"sk-e30aaee8bcce40348cf790585352bf36",
    "base_url":"https://dashscope.aliyuncs.com/compatible-mode/v1",
    "is_steam":False
}


def translate():

    w = tk.winfo_width()
    h = tk.winfo_height()

    x = tk.winfo_x()
    y = tk.winfo_y() - h 
    
    # ocr 识别文字
    bbox=(x, y, x+w, y+h)
    screenshot = ImageGrab.grab(bbox=bbox)

    img_byte_arr = io.BytesIO()
    screenshot.save(img_byte_arr, format='PNG')
    img_base64 = base64.b64encode(img_byte_arr.getvalue()).decode("utf-8")

    pic_txt= api_ocr.talk_img(need_ocr,img_base64)
    # label_1.configure(text=pic_txt)
    
    # 如果扫描的句子变了，才重新翻译
    if len(old_txt)==0:
        old_txt.append(pic_txt)
    else:
        if pic_txt != old_txt[0]:
            # translate
            answer = api_translate.talk(need_tran + pic_txt)
            label.configure(text=answer)
            old_txt[0] = pic_txt


    tk.after(200,translate)


tk = Tk()
tk.geometry('1000x200')
tk.attributes('-topmost', True)
tk.title(' ')

# label_1 = Label(tk,text = 'ok',font=("fixed",15))
# label_1.pack(expand=YES)

label = Label(tk,text = 'hi',font=("fixed",15))
label.pack()    # expand=YES

api_ocr = Api(qwen_vl_ocr_plus_api,"你是一个识别文字的专家")
need_ocr = "请输出图片里面的文字"

api_translate = Api(qwen_plus_api,"你是一个翻译助手")
need_tran = "请有感情地翻译图片里面的文字，翻译后输出中文，注意：不要有多余的句子："


old_txt = []

tk.after(1000,translate)

tk.mainloop()



